# KBO 야구 특화 챗봇 

이 노트북은 논문 "A Chatbot for Football Analytics" 아키텍처를 KBO 야구에 적용한 챗봇의 전체 파이프라인을 실행합니다.

## 주요 구성 요소
1. **데이터 적재 (Ingest)**: JSON → ChromaDB 벡터 저장소
2. **쿼리 분류 (Classifier)**: general / season_analysis / match_analysis
3. **하이브리드 검색 (Retriever)**: Semantic + BM25
4. **응답 생성 (Chain)**: LLM 기반 분석
5. **에이전트 (Agent)**: Function Calling 지원

---

## 1. 환경 설정 및 라이브러리 로드

환경 변수를 `.env` 파일에서 로드하고, API 키가 안전하게 관리되는지 확인합니다.

In [1]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

# 프로젝트 경로 설정
PROJECT_ROOT = Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT))

# API 키 확인 (값은 출력하지 않음 - 보안)
api_key = os.getenv("OPENAI_API_KEY")
print(f"✅ 프로젝트 루트: {PROJECT_ROOT}")
print(f"✅ OPENAI_API_KEY 로드: {'성공 ✓' if api_key else '실패 ✗ (.env 파일 확인 필요)'}")

if not api_key:
    print("\n⚠️ .env 파일에 OPENAI_API_KEY를 설정하세요:")
    print("   OPENAI_API_KEY=sk-your-api-key-here")

✅ 프로젝트 루트: c:\Coding\PAINS-LLM\PAINS-LLM
✅ OPENAI_API_KEY 로드: 성공 ✓


## 2. 프로젝트 파일 구조 분석

현재 프로젝트의 파일 구조를 트리 형태로 출력합니다.

In [2]:
def print_tree(path: Path, prefix: str = "", ignore: set = None):
    """디렉토리 구조를 트리 형태로 출력"""
    if ignore is None:
        ignore = {'.git', '__pycache__', '.venv', 'node_modules', '.vscode', 'chroma_db'}
    
    items = sorted(path.iterdir(), key=lambda x: (x.is_file(), x.name))
    items = [i for i in items if i.name not in ignore]
    
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        current_prefix = "└── " if is_last else "├── "
        print(f"{prefix}{current_prefix}{item.name}")
        
        if item.is_dir():
            next_prefix = "    " if is_last else "│   "
            print_tree(item, prefix + next_prefix, ignore)

print("📁 프로젝트 구조:\n")
print_tree(PROJECT_ROOT)

📁 프로젝트 구조:

├── data
│   └── raw
│       ├── matches
│       │   ├── 2025_POST_MATCH_PITCHING_STATS.json
│       │   └── 2025_REGULAR_MATCH_PITCHING_STATS.json
│       └── seasons
│           ├── 2025_POST_PITCHING_STATS.json
│           └── 2025_REGULAR_PITCHING_STATS.json
├── src
│   ├── __init__.py
│   ├── agent.py
│   ├── chain.py
│   ├── classifier.py
│   ├── config.py
│   ├── ingest.py
│   ├── retriever.py
│   ├── tools.py
│   └── utils.py
├── .env
├── .gitignore
├── A Chatbot for Football Analytics A deep dive into RAG, LLM Orchestration and Function Calling_extracted.txt
├── README.md
├── main.ipynb
├── main.py
└── requirements.txt


## 3. 핵심 모듈 임포트

`src/` 디렉토리의 핵심 모듈들을 임포트합니다.

In [3]:
# 핵심 모듈 임포트
from src.config import (
    CHROMA_DB_DIR, COLLECTION_NAME, EMBEDDING_MODEL,
    SEASON_DATA_DIR, MATCH_DATA_DIR, LLM_MODEL, RETRIEVAL_TOP_K
)

print("📦 설정 정보:")
print(f"   - 임베딩 모델: {EMBEDDING_MODEL}")
print(f"   - LLM 모델: {LLM_MODEL}")
print(f"   - ChromaDB 경로: {CHROMA_DB_DIR}")
print(f"   - 컬렉션 이름: {COLLECTION_NAME}")
print(f"   - 시즌 데이터: {SEASON_DATA_DIR}")
print(f"   - 경기 데이터: {MATCH_DATA_DIR}")
print(f"   - Top-K 검색: {RETRIEVAL_TOP_K}")

📦 설정 정보:
   - 임베딩 모델: intfloat/multilingual-e5-large-instruct
   - LLM 모델: gpt-4o
   - ChromaDB 경로: c:\Coding\PAINS-LLM\PAINS-LLM\data\chroma_db
   - 컬렉션 이름: kbo_data
   - 시즌 데이터: c:\Coding\PAINS-LLM\PAINS-LLM\data\raw\seasons
   - 경기 데이터: c:\Coding\PAINS-LLM\PAINS-LLM\data\raw\matches
   - Top-K 검색: 5


## 4. 데이터 적재 (Ingest)

JSON 데이터를 ChromaDB 벡터 저장소에 적재합니다.

### 핵심 전략 (논문 Section 4.2)
- JSON을 그대로 임베딩하지 않음
- **서술형 문장(Descriptive Sentence)** 을 생성하여 임베딩
- 원본 데이터는 metadata에 저장

In [4]:
import time
from src.ingest import ingest_all_data, initialize_vector_store

# 데이터 적재 실행
# clear_existing=True: 기존 DB 삭제 후 재적재
print("⏱️ 데이터 적재 시작... (CPU 임베딩은 시간이 걸릴 수 있습니다)")
start_time = time.time()

result = ingest_all_data(clear_existing=True)

elapsed_time = time.time() - start_time

if result:
    doc_count = result._collection.count()
    print(f"\n🎉 벡터 스토어 준비 완료!")
    print(f"   - 문서 수: {doc_count}")
    print(f"   - 소요 시간: {elapsed_time:.1f}초")
    print(f"   - 문서당 평균: {elapsed_time/doc_count*1000:.1f}ms")

⏱️ 데이터 적재 시작... (CPU 임베딩은 시간이 걸릴 수 있습니다)
🚀 KBO 데이터 적재 파이프라인 시작
🗑️ 기존 벡터 스토어 삭제: c:\Coding\PAINS-LLM\PAINS-LLM\data\chroma_db

📥 데이터 로드 중...
📂 시즌 데이터 파일 발견: 2개
   - 2025_POST_PITCHING_STATS.json: 59개 선수 레코드 발견
   - 2025_REGULAR_PITCHING_STATS.json: 277개 선수 레코드 발견
📂 경기 데이터 파일 발견: 2개
   - 2025_POST_MATCH_PITCHING_STATS.json: 168개 레코드 발견
   - 2025_REGULAR_MATCH_PITCHING_STATS.json: 7077개 레코드 발견

📊 로드된 데이터 요약:
   - 시즌 데이터: 336건
   - 경기 데이터: 7245건

🔄 시즌 데이터 변환 중...


Season: 100%|██████████| 336/336 [00:00<00:00, 28060.57it/s]



🔄 경기 데이터 변환 중...


Match: 100%|██████████| 7245/7245 [00:00<00:00, 40966.52it/s]
c:\Coding\PAINS-LLM\PAINS-LLM\src\ingest.py:48: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(



📦 ChromaDB 초기화 중... (문서 수: 7581)
   배치 크기: 100개씩 처리


ValueError: Expected metadata value to be a str, int, float or bool, got ['Samsung'] which is a list in upsert.

Try filtering complex metadata from the document using langchain_community.vectorstores.utils.filter_complex_metadata.

## 5. 쿼리 분류 테스트 (Classifier)

사용자 쿼리를 `general`, `season_analysis`, `match_analysis` 중 하나로 분류합니다.

In [ ]:
from src.classifier import classify_query

# 테스트 쿼리들
test_queries = [
    "야구 규칙 설명해줘",              # general
    "한화 이글스 올시즌 성적 분석해줘",   # season_analysis
    "어제 한화 LG 경기 어땠어?",        # match_analysis
]

print("🔍 쿼리 분류 테스트:\n")
for query in test_queries:
    result = classify_query(query)
    print(f"쿼리: {query}")
    print(f"  → 분류: {result.query_type}")
    print(f"  → 팀: {result.teams}")
    print(f"  → 신뢰도: {result.confidence:.0%}")
    print()

## 6. 하이브리드 검색 테스트 (Retriever)

Semantic Search + BM25를 결합한 하이브리드 검색을 수행합니다.

In [ ]:
from src.retriever import retrieve_for_query

# 검색 테스트
test_query = "한화 이글스 시즌 성적"

print(f"🔎 검색 쿼리: '{test_query}'\n")
results = retrieve_for_query(test_query, top_k=3)

for i, doc in enumerate(results, 1):
    print(f"결과 {i}:")
    print(f"  타입: {doc.metadata.get('type')}")
    print(f"  팀: {doc.metadata.get('teams', doc.metadata.get('team'))}")
    print(f"  내용: {doc.page_content[:100]}...")
    print()

## 7. 전체 파이프라인 실행 (Chain)

분류 → 검색 → 생성의 전체 파이프라인을 실행합니다.

In [ ]:
from src.chain import run_analysis

# 분석 실행
query = "한화 이글스 올시즌 성적 분석해줘"
print(f"💬 질문: {query}\n")

result = run_analysis(query)

print(f"📊 분류: {result.query_type}")
print(f"📊 팀: {result.teams}")
print(f"📊 검색 점수: {result.retrieval_score:.2%}")
print(f"📊 대시보드 필요: {result.needs_dashboard}")
print(f"\n🤖 응답:\n{result.response[:500]}...")

## 8. 대화형 에이전트 (Agent)

Function Calling을 지원하는 대화형 에이전트를 사용합니다.

In [ ]:
from src.agent import chat

# 에이전트 대화 테스트
queries = [
    "안녕! KBO에 대해 알려줘",
    "한화 이글스 올시즌 성적 분석해줘",
]

for q in queries:
    print(f"👤 User: {q}")
    response = chat(q)
    print(f"🤖 Assistant: {response.response[:300]}...")
    if response.dashboard:
        print(f"   📊 대시보드 생성됨!")
    print("-" * 50)

---

## 📝 논문 대비 구현 차이점 및 ChromaDB 구성

### 논문과 동일하게 구현한 부분
| 논문 섹션 | 구현 내용 |
|----------|----------|
| Section 4.2 (Embedding) | `multilingual-e5-large-instruct` 모델 사용, L2 정규화 |
| Section 4.3.1 (Query Classification) | CoT 프롬프팅으로 general/season/match 분류 |
| Section 4.3.2 (Query Normalization) | RapidFuzz QRatio 스코러로 팀명 퍼지 매칭 |
| Section 4.4.1 (Instruct Format) | `"Instruct: ... Query: ..."` 포맷 사용 |
| Section 4.4.2 (Hybrid Retrieval) | Semantic (0.8) + BM25 (0.2) 앙상블 |
| Section 4.5.4 (Function Calling) | 대시보드 생성용 도구 정의 |

### 논문과 다르게 (KBO에 맞게) 수정한 부분

| 항목 | 논문 (축구) | 이 프로젝트 (야구) |
|-----|-----------|------------------|
| **도메인** | 유럽 축구 리그 | KBO 한국 프로야구 |
| **TEAM_MAP** | 축구팀 + 대회 매핑 | 10개 KBO 팀 + 한글 별칭 |
| **데이터 구조** | 리그/경기 JSON | 시즌/경기 JSON (야구 스탯) |
| **KPI** | xG, 패스 등 | 타율, 방어율, OPS 등 |
| **Instruct 프롬프트** | "football dataset" | "baseball dataset" |

### ChromaDB 벡터 저장소 구성

```
data/
└── chroma_db/           ← 논문과 동일하게 로컬 영구 저장소 사용
    ├── chroma.sqlite3   ← 메타데이터 저장
    └── [UUID 폴더들]/   ← 벡터 임베딩 저장
```

- **저장 위치**: `data/chroma_db/` (config.py의 `CHROMA_DB_DIR`)
- **컬렉션 이름**: `kbo_data` (config.py의 `COLLECTION_NAME`)
- **임베딩 차원**: 1024 (multilingual-e5-large-instruct)
- **인덱스 방식**: HNSW (ChromaDB 기본값)

> 💡 논문에서는 PlaymakerAI API에서 데이터를 가져왔지만, 이 프로젝트는 로컬 JSON 파일을 사용합니다.
> 이는 API 없이도 테스트할 수 있도록 한 설계 결정입니다.

In [ ]:
# ChromaDB 상태 확인
from src.ingest import initialize_vector_store

vector_store = initialize_vector_store(documents=None)  # 기존 스토어 로드
collection = vector_store._collection

print("🗄️ ChromaDB 벡터 저장소 정보:")
print(f"   - 저장 경로: {CHROMA_DB_DIR}")
print(f"   - 컬렉션 이름: {collection.name}")
print(f"   - 문서 수: {collection.count()}")
print(f"   - 임베딩 모델: {EMBEDDING_MODEL}")
print(f"   - 임베딩 차원: 1024 (multilingual-e5-large-instruct)")